# Your First Binary Simulations with POSYDON 🌠

**Tutorial goal:**

In this tutorial, we will run a small population of 10 binaries locally, and explore how to manipulate the output data from your population.

**New concepts:**

- Population ini file

If you haven't done so yet, export the path POSYDON environment variables.
Set these parameters in your `.bash_profile` or `.zshrc` if you use POSYDON regularly.

In [ ]:
%env PATH_TO_POSYDON=/YOUR/POSYDON/PATH/
%env PATH_TO_POSYDON_DATA=/YOUR/POSYDON_DATA/PATH/

## Creating the Initialisation File

----
To run population synthesis with POSYDON, a `population_params.ini` file is required.
This file described how the stellar population is created and what prescriptions and parameters are implemented in specific steps.

POSYDON comes with a default `population_params_default.ini` file found at `PATH_TO_POSYDON/posydon/popsyn` or have a look [here](population_params.ini).

The file is split in three main parts. You can find more details about their properties by clicking on their links.
1. **[SimulationProperties]():**
   - these describe the properties and parameters of different steps in the evolution of a binary systems.
2. **[BinaryPopulation]():** 
   - parameters of the initial sampling of the binary population, such as initial mass function, period distribution, and metallicity.
   - Also contains parameters on how the population is ran, such as how many binaries are kept in memory.
3. **[SavingOutput]()**
   - Describes the data from the binary and each individual star to the output files.

We will copy the default population run parameter file to the current folder.

In [2]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'

# Creating and running a binary population

The copied `population_params.ini` contains the parameters to run 10 binaries at a metallicity of $Z=10^{-4}$.

If you open the file and scroll down to the **BinaryPopulation** section, you will see how they're defined:

```
metallicity = [0.0001] # [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
# In units of solar metallicity
...
number_of_binaries = 10
# int
```

If you like to run a small population in a notebook, you can use the `PopulationRunner` to do this. If you want to run a specific binary instead, have a look at the [Binary Tutorial]().

The `PopulationRunner` class takes the `.ini` file and sets-up a population run.

This will create [BinaryPopulations]() for each metallicity defined in the `.ini` file.
In this case, we can check and see that a single BinaryPopulation is created and contains 10 binaries.

In [3]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini')

In [4]:
print('Number of binary populations:',len(poprun.binary_populations))
print('Metallicity:', poprun.binary_populations[0].metallicity)
print('Number of binaries:', poprun.binary_populations[0].number_of_binaries)  

Number of binary populations: 1
Metallicity: 0.0001
Number of binaries: 10


After setting up the population run, we can evolve it. This should take about 20 seconds, but depends on your machine.

In [5]:
poprun.evolve()

# Inspecting the population: Population class

When you ran the population, you might have seen that a temporary folder with the name `1e-04_Zsun_batches` was created while the binaries were evolved.
This is a temporary folder in which populations are temporarly saved.
After the binary evolution has finished, the binaries in the folder are moved to a single file named `1e-04_Zsun_popululation.h5`. This is done automatically, when you run a population using the `PopulationRunner` class.

The created file contains 3 main components:

1. **history:** the evolution of an individual binary in a pandas DataFrame
2. **oneline:** a single line to describe the initial and final conditions and some one-of parameters, such as the metallicity.
3. **mass_per_metallicity:** some metadata on the population, such as the total simulated mass, the actual underlying mass of the population, and the number of binaries in the file.

The `Population` provides an interface with these components in the file, such that you're able to share the populations runs and can work with large population that do not fit in memory.


<div class="alert alert-block alert-warning"><b>Older Population Files</b> 

If you're using older population files, you can make them compatible with the `Population` class by calling `Population(pop_file, metallicity,ini_file)`, where the `metallicity` is in solar units. You will only need to do this once; afterwards you can initialise the class like normal.<div>

In [6]:
from posydon.popsyn.synthetic_population import Population

In [7]:
pop = Population('1e-04_Zsun_population.h5')

The `pop.mas_per_met` shows some basic information about the population you've just created.


1. The index is the metallicity of your population in solar units.
2. **simulated mass** is the total ZAMS mass that has been evolved in the population.
3. **underlying mass** is the actual mass of the population if one integrates the IMF and period distribution fully.
4. **number_of_systems** shows the 10 systems in the file.

In [8]:
pop.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,296.055678,1664.965703,10


There are some additional metadata properties available, such as:
- `metallicities`: the metallicity in absolute metallicity
- `solar_metallicities`: the metallicities in the file in solar metallicity
- `number_of_systems`: the total number of systems in the Population file
- `indices`: the indices of the binaries in the file
- `columns`: the columns available in the `history` and `oneline` dataframes

In [9]:
# you can also access the total number of systems in the file with
print(pop.number_of_systems)

10


## Population.history

`pop.history` loads in the full history of all the binaries into memory.
You can access individual or a selections of the population using several methods:


1. pop.history[5]
2. pop.history[[0,4]]
3. pop.history['time]
4. pop.history.select()

The select function is the most powerfull way to access the binaries, because it allows you to perform selections based on the columns available in the history dataframe.

For example, below we can select on `state == 'RLO1'`, which gives us all the rows with RLO1 occuring.
At the same time, we can request only specific columns, 


In [10]:
# select only binary_index 5
pop.history[5]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.000000e+00,587.359193,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,10.343498,...,NaN,NaN,NaN,NaN,0.750999,0.249000,NaN,NaN,NaN,NaN
5,RLO1,oCE1,2.538854e+07,379.015276,0.0,-1.323763,step_HMS_HMS,0.009525,H-rich_Central_He_depleted,9.804626,...,0.0,0.0,0.0,0.0,0.712405,0.287593,0.750872,0.249093,0.986846,4.662131
5,merged,oMerging1,2.538854e+07,379.015276,0.0,-1.323763,step_CE,0.000033,H-rich_Central_He_depleted,9.804626,...,0.0,0.0,0.0,0.0,0.712405,0.287593,0.750872,0.249093,0.986846,4.662131
5,merged,CC1,2.540137e+07,NaN,NaN,NaN,step_merged,0.128643,H-rich_Central_C_depletion,11.581089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,merged,NaN,2.540137e+07,NaN,NaN,NaN,step_SN,0.000501,NS,1.418336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,merged,END,2.540137e+07,NaN,NaN,NaN,step_end,0.000019,NS,1.418336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# select binary 0 and 4
pop.history[[0,4]]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,2.444245e+00,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,34.916452,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
0,RLO1,CC1,6.107385e+06,3.801710e+00,0.000000,-4.169659,step_HMS_HMS,0.753225,H-rich_Central_C_depletion,18.813159,...,0.000000,0.000000,0.000000,0.000000,0.218835,7.811632e-01,0.231017,0.768982,0.661030,9.628504
0,detached,NaN,6.107385e+06,3.675714e+00,0.039697,NaN,step_SN,0.000466,BH,16.062818,...,0.000000,0.000000,0.000000,0.000000,0.218835,7.811632e-01,0.231017,0.768982,0.661030,9.628504
0,RLO2,oRLO2,6.947304e+06,4.053375e+00,0.000000,NaN,step_detached,2.026444,BH,16.062818,...,0.000000,NaN,0.000000,0.000000,0.024994,9.750047e-01,0.750999,0.248974,0.336814,3.438405
0,RLO2,CC2,7.461656e+06,2.447725e+00,0.000000,-6.741603,step_CO_HMS_RLO,0.094441,BH,16.391644,...,18.085137,0.253985,16.386845,0.155071,0.000000,1.352954e-12,0.223521,0.776478,0.521943,0.194875
0,detached,NaN,7.461656e+06,2.580692e+00,0.027430,NaN,step_SN,0.000457,BH,16.391644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021119
0,contact,CO_contact,6.613385e+09,3.444628e-08,0.000000,NaN,step_dco,0.294264,BH,16.391644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021119
0,contact,END,6.613385e+09,3.444628e-08,0.000000,NaN,step_end,0.000023,BH,16.391644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021119
4,detached,ZAMS,0.000000e+00,8.717908e-01,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,23.995709,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN


In [12]:
pop.history['time']

,time
binary_index,
0,0.000000e+00
0,6.107385e+06
0,6.107385e+06
0,6.947304e+06
0,7.461656e+06
...,...
9,6.105044e+07
9,6.105616e+07
9,6.105616e+07


You can also check what columns are available in the history file:

In [13]:
pop.history.columns

Index(['state', 'event', 'time', 'orbital_period', 'eccentricity',
       'lg_mtransfer_rate', 'step_names', 'step_times', 'S1_state', 'S1_mass',
       'S1_log_R', 'S1_log_L', 'S1_lg_mdot', 'S1_he_core_mass',
       'S1_he_core_radius', 'S1_co_core_mass', 'S1_co_core_radius',
       'S1_center_h1', 'S1_center_he4', 'S1_surface_h1', 'S1_surface_he4',
       'S1_surf_avg_omega_div_omega_crit', 'S1_spin', 'S2_state', 'S2_mass',
       'S2_log_R', 'S2_log_L', 'S2_lg_mdot', 'S2_he_core_mass',
       'S2_he_core_radius', 'S2_co_core_mass', 'S2_co_core_radius',
       'S2_center_h1', 'S2_center_he4', 'S2_surface_h1', 'S2_surface_he4',
       'S2_surf_avg_omega_div_omega_crit', 'S2_spin'],
      dtype='object')

In [14]:
# using the select function
pop.history.select(where='index == 9')
# selecting all RLO1 states and only time and state columns
pop.history.select(where='state == RLO1', columns=['time', 'state'])
# selecting rows 10 to 16
pop.history.select(start=10, stop=16)


,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
1,merged,oMerging1,3.136291e+07,0.908964,0.0,-1.428134,step_CE,0.000041,H-rich_Core_He_burning,8.360338,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.735616,0.264379,0.750970,0.248991,0.989370,10.469333
1,merged,CC1,3.278411e+07,NaN,NaN,NaN,step_merged,1.998618,H-rich_Central_C_depletion,10.129632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,NaN,3.278411e+07,NaN,NaN,NaN,step_SN,0.000463,NS,1.415666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,END,3.278411e+07,NaN,NaN,NaN,step_end,0.000021,NS,1.415666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,detached,ZAMS,0.000000e+00,147.393033,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,42.424175,...,NaN,NaN,NaN,NaN,0.750999,0.249000,NaN,NaN,NaN,NaN
2,RLO1,CC1,5.261878e+06,89.218216,0.0,-6.174442,step_HMS_HMS,0.008554,H-rich_Central_C_depletion,26.855729,...,1.411542e-14,2.053381e-16,1.365520e-14,1.239204e-16,0.499341,0.500657,0.413748,0.586249,0.902543,13.521817


You might have notices while using the above functions that not all the binaries will have the same length in the history.
You can access these with `pop.history_lengths`. This information is also stored in the population file.

In [15]:
pop.history_lengths

,index
index,
0,8
1,6
2,7
3,6
4,6
5,6
6,6
7,6
8,6


## Population.oneline

`Population.oneline` provides a similar interface to accessing the DataFrame in the population file as `Population.history`, with similar functionality being available.

In [16]:
pop.oneline[5]
pop.oneline[[0,4]]
pop.oneline.select(where='index == 9')
pop.oneline.select(where='S1_mass_i > 20', columns=['state_i','S1_state_i', 'S1_mass_i', 'S2_state_i','S2_mass_i'])

,state_i,S1_state_i,S1_mass_i,S2_state_i,S2_mass_i
binary_index,,,,,
0,detached,H-rich_Core_H_burning,34.916452,H-rich_Core_H_burning,29.754275
2,detached,H-rich_Core_H_burning,42.424175,H-rich_Core_H_burning,19.612658
4,detached,H-rich_Core_H_burning,23.995709,H-rich_Core_H_burning,16.080349
7,detached,H-rich_Core_H_burning,35.370320,H-rich_Core_H_burning,20.640103


In [17]:
pop.oneline.columns

Index(['state_i', 'event_i', 'time_i', 'orbital_period_i', 'eccentricity_i',
       'lg_mtransfer_rate_i', 'step_names_i', 'step_times_i', 'state_f',
       'event_f', 'time_f', 'orbital_period_f', 'eccentricity_f',
       'lg_mtransfer_rate_f', 'step_names_f', 'step_times_f', 'S1_state_i',
       'S1_mass_i', 'S1_log_R_i', 'S1_log_L_i', 'S1_lg_mdot_i',
       'S1_he_core_mass_i', 'S1_he_core_radius_i', 'S1_co_core_mass_i',
       'S1_co_core_radius_i', 'S1_center_h1_i', 'S1_center_he4_i',
       'S1_surface_h1_i', 'S1_surface_he4_i',
       'S1_surf_avg_omega_div_omega_crit_i', 'S1_spin_i', 'S1_state_f',
       'S1_mass_f', 'S1_log_R_f', 'S1_log_L_f', 'S1_lg_mdot_f',
       'S1_he_core_mass_f', 'S1_he_core_radius_f', 'S1_co_core_mass_f',
       'S1_co_core_radius_f', 'S1_center_h1_f', 'S1_center_he4_f',
       'S1_surface_h1_f', 'S1_surface_he4_f',
       'S1_surf_avg_omega_div_omega_crit_f', 'S1_spin_f',
       'S1_natal_kick_array_0', 'S1_natal_kick_array_1',
       'S1_natal_kick_a

## Population.formation_channels

You might be interested in figuring out what sort of formation pathways/channels a binary has followed through its evolution.

This is not a standard output of the population synthesis, but you can include it into the population file by calculating it. 
If you would like more detail on the initial mass transfer, you can set `mt_history=True`.

This will write the formation channels to the Population file, which can be accessed by `Population.formation_channels`.


In [18]:
pop.calculate_formation_channels(mt_history=True)

In [19]:
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_oRLO2_CC2_CO_contact_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_oRLO2_CC2_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
3,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END


Next time you open this population file, the `formation_channels` will be available without having to be recalculated.

In [20]:
pop = Population('1e-04_Zsun_population.h5')
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_oRLO2_CC2_CO_contact_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_oRLO2_CC2_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
3,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END


## Selecting a sub-population

You might just want a small sub-selection of the full population, especially if you're working with large population and multi-metallicity runs.

The `Population.export_selection()` function will export just the indices of the binaries you're interested in into a new file.
The simulated and underlying mass will remain the same, since they are dependent on the population run.

If we select just 2 binaries and export them, we create a new population of just the binaries you're interested in.
In the [BBH analysis]() and [GRB analysis]() tutorials, we show how to perform a selection with multiple criteria and metallicities.



In [23]:
indices = [0,9]
pop.export_selection(indices, 'selected.h5')    

In [24]:
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,296.055678,1664.965703,2


By default this export-selection will overwrite any file already present and create a new ouput file.

However, if you set overwrite=False, you can export multiple populations together into a single file.
This is especially useful when creating multi-metallicity populations.


In [26]:
# this will overwrite the file
pop.export_selection(indices, 'selected.h5')

selected = Population('selected.h5')
selected.mass_per_metallicity


,simulated_mass,underlying_mass,number_of_systems
0.0001,296.055678,1664.965703,2


In [27]:
# This will add to the file
pop.export_selection(indices, 'selected.h5', overwrite=False)
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,592.111356,3329.931407,4


<br/><br/>

Feel free to explore the small binary population you've just created!

If you want to learn more about population synthesis and how to build more complex models it is advised to continue with the remaining tutorials and consult the POSYDON documentation.

### Local MPI runs

To speed up population synthesis runs, you can run on a computing cluster, as described in [HPC Facilities](pop_syn), or you can distribute the population synthesis across multiple cores on your local machine using MPI.

To enable local MPI runs, go into the `population_params.ini`  and change `use_MPI` to `True`.

It's important to note that you cannot run have this option enabled for cluster runs!

We create a binary population simulation script to run the population:

In [ ]:
%%writefile script.py
from posydon.popsyn.synthetic_population import PopulationRunner

if __name__ == "__main__":
    synth_pop = PopulationRunner("./population_params.ini")
    synth_pop.evolve()

This script can be initiated using a local where `NR_processors` is the number of processors you would like to us.

In [ ]:
mpiexec -n ${NR_processors} python script.py

This will create a folder for each metallicity in the population and store output of the parallel runs in it.

You will have to concatenate these runs manually into a single population file per metallicity, which can be achieved using the following code:

In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner

synth_pop = PopulationRunner("./population_params.ini")
for pop in synth_pop.binary_populations:
    synth_pop.merge_parallel_runs(pop)